### Imports

In [1]:
import numpy as np
import xarray as xr
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import seaborn
import os
import glob

import pandas as pd

### Functions

In [2]:
def find_nearest(array,value):
    idx,val = min(enumerate(array), key=lambda x: abs(x[1]-value))
    return idx

### Define paths and read data

In [3]:
# Paths and ensemble members
EXP="HIST2"
OP="/gws/nopw/j04/canari/shared/large-ensemble/derived/"+EXP+"/"
ENS=sorted(os.listdir(OP))

In [4]:
# List of lists of files
EM_moc_files=[]
EM_mocsig_files=[]
for i in ENS:
    EM_moc_files.append(glob.glob(OP+i+"/OCN/yearly/*/*_moc.nc"))
    EM_mocsig_files.append(glob.glob(OP+i+"/OCN/yearly/*/*_mocsig.nc"))

In [5]:
# Read data into an xarray dataset with new dimension 'ensemble_member'
EM_mocsig = xr.open_mfdataset(EM_mocsig_files[0])
EM_mocsig['ensemble_member'] = int(ENS[0])
EM_mocsig = EM_mocsig.expand_dims(dim='ensemble_member').set_coords('ensemble_member')

for i in range(1,len(ENS)):
    ds = xr.open_mfdataset(EM_mocsig_files[i])
    ds['ensemble_member'] = int(ENS[i])
    ds = ds.expand_dims(dim='ensemble_member').set_coords('ensemble_member')
    EM_mocsig = xr.concat([EM_mocsig,ds],'ensemble_member')

### EM_moc is now a single dataset which looks like this...

In [21]:
EM_mocsig

<xarray.Dataset>
Dimensions:          (sigma: 158, time_counter: 780, ensemble_member: 20,
                      y: 1207, x: 1)
Coordinates:
  * sigma            (sigma) float32 30.02 30.08 30.12 ... 37.78 37.83 37.88
  * time_counter     (time_counter) object 1950-01-16 00:00:00 ... 2014-12-16...
  * ensemble_member  (ensemble_member) int64 1 10 11 12 13 18 2 ... 31 4 5 6 7 9
Dimensions without coordinates: y, x
Data variables:
    nav_lon          (ensemble_member, time_counter, y, x) float32 dask.array<chunksize=(1, 12, 1207, 1), meta=np.ndarray>
    nav_lat          (ensemble_member, time_counter, y, x) float32 dask.array<chunksize=(1, 12, 1207, 1), meta=np.ndarray>
    zomsfglo         (ensemble_member, time_counter, sigma, y, x) float32 dask.array<chunksize=(1, 12, 158, 1207, 1), meta=np.ndarray>
    zomsfatl         (ensemble_member, time_counter, sigma, y, x) float32 dask.array<chunksize=(1, 12, 158, 1207, 1), meta=np.ndarray>
    zomsfinp         (ensemble_member, time_counter, sigma, y, x) float32 dask.array<chunksize=(1, 12, 158, 1207, 1), meta=np.ndarray>
    zomsfind         (ensemble_member, time_counter, sigma, y, x) float32 dask.array<chunksize=(1, 12, 158, 1207, 1), meta=np.ndarray>
    zomsfpac         (ensemble_member, time_counter, sigma, y, x) float32 dask.array<chunksize=(1, 12, 158, 1207, 1), meta=np.ndarray>
    zoisoglo         (ensemble_member, time_counter, sigma, y, x) float32 dask.array<chunksize=(1, 12, 158, 1207, 1), meta=np.ndarray>
    zoisoatl         (ensemble_member, time_counter, sigma, y, x) float32 dask.array<chunksize=(1, 12, 158, 1207, 1), meta=np.ndarray>
    zoisoinp         (ensemble_member, time_counter, sigma, y, x) float32 dask.array<chunksize=(1, 12, 158, 1207, 1), meta=np.ndarray>
    zoisoind         (ensemble_member, time_counter, sigma, y, x) float32 dask.array<chunksize=(1, 12, 158, 1207, 1), meta=np.ndarray>
    zoisopac         (ensemble_member, time_counter, sigma, y, x) float32 dask.array<chunksize=(1, 12, 158, 1207, 1), meta=np.ndarray>
Attributes:
    start_date:        -1
    output_frequency:  N/A
    CONFIG:            N/A
    CASE:              N/A
    history:           Wed Mar  6 08:09:53 2024: ncks -O --no_abc --4 --dfl_l...
    NCO:               netCDF Operators version 5.1.5 (Homepage = http://nco....

In [20]:
# Size of dataset in GB
EM_mocsig.nbytes/1024**3

110.96818803995848

### Find 26N and 1km, extract AMOC@26N

In [22]:
y26 = find_nearest(EM_mocsig['nav_lat'][0,0,:,0],26)
y42 = find_nearest(EM_mocsig['nav_lat'][0,0,:,0],42)
y50 = find_nearest(EM_mocsig['nav_lat'][0,0,:,0],50)
y55 = find_nearest(EM_mocsig['nav_lat'][0,0,:,0],55)
#z1k = find_nearest(EM_mocsig['depthw'],1000)
EM_amocsig_26N = EM_mocsig['zomsfatl'].isel(y=y26).compute()
EM_amocsig_42N = EM_mocsig['zomsfatl'].isel(y=y42).compute()
EM_amocsig_50N = EM_mocsig['zomsfatl'].isel(y=y50).compute()
EM_amocsig_55N = EM_mocsig['zomsfatl'].isel(y=y55).compute()

#EM_amocsig_26N_1k = EM_amocsig_26N.isel(depthw=z1k)

### Write extracted AMOC variables - small files for ease of use

In [23]:
EM_amocsig_26N.to_netcdf(OP+"../Ensemble_amoc_s_26N.nc")
EM_amocsig_42N.to_netcdf(OP+"../Ensemble_amoc_s_42N.nc")
EM_amocsig_50N.to_netcdf(OP+"../Ensemble_amoc_s_50N.nc")
EM_amocsig_55N.to_netcdf(OP+"../Ensemble_amoc_s_55N.nc")